# 14. 이진화

## Threshold

- 임계값을 기준으로 흑/백으로 분류하는 함수
- image : 처리할 Gray Scale 이미지
- thresh : 임계값(전체 픽셀에 적용)
- max_value : 임계값을 넘었을 때 적용할 값
- type : 임계점을 처리하는 방식

- THRESH_BINARY : 임계값보다 크면 max_value, 작으면 0
- THRESH_BINARY_INV : 임계값보다 작으면 max_value, 크면 0
- THRESH_TRUNC : 임계값보다 크면 임계값, 작으면 그대로
- THRESH_TOZERO : 임계값보다 크면 그대로, 작으면 0
- THRESH_TOZERO_INV : 임계값보다 크면 0, 작으면 그대로

cv2.threshold(image, thresh, max_value, type):

In [3]:
import cv2
image = cv2.imread('./data/gray_image.jpeg', cv2.IMREAD_GRAYSCALE)

images = []
# 임계값 127
ret, thres1 = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
ret, thres2 = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
ret, thres3 = cv2.threshold(image, 127, 255, cv2.THRESH_TRUNC)
ret, thres4 = cv2.threshold(image, 127, 255, cv2.THRESH_TOZERO)
ret, thres5 = cv2.threshold(image, 127, 255, cv2.THRESH_TOZERO_INV)

images.append(thres1)
images.append(thres2)
images.append(thres3)
images.append(thres4)
images.append(thres5)

for i, pic in enumerate(images):
    cv2.imshow(f'gray_{i}', cv2.cvtColor(pic, cv2.COLOR_GRAY2RGB))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2
img = cv2.imread('./data/apple.png', cv2.IMREAD_COLOR)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 임계값 127
## 127보다 크면 255(하양)
## 127보다 작으면 0(검정)

ret, binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY_INV)

cv2.imshow('img', img)
cv2.imshow('gray', img_gray)
cv2.imshow('binary', binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2

def empty(pos):
    # print(pos)
    pass

img = cv2.imread('./data/apple.png', cv2.IMREAD_GRAYSCALE)

name = 'Trackbar'
cv2.namedWindow(name)

cv2.createTrackbar('threshold', name, 127, 255, empty) # bar이름, 창의 이름, 초기값, 최대값, 이벤트처리

while True:
    thresh = cv2.getTrackbarPos('threshold', name) # bar이름, 창의 이름
    ret, binary = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)

    if not ret:
        break

    cv2.imshow(name, binary)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

## Adaptive Threshold

이미지를 작은 영역으로 나누어서 임계치 적용

적응 임계점 처리 함수
- 하나의 이미지에 다수의 조명 상태가 존재하는 경우 적용하면 좋음
- Adaptive Threshold를 이용하면, 전체 픽셀을 기준으로 임계값을 적용하지 않음
- max_value : 임계값을 넘었을때 적용할 값
- adaptive_method : 임계값을 결정하는 계산방법
- ADAPTIVE_THRESH_MEAN_C : 주변영역의 평균값으로 결정

- ADAPTIVE_THRESH_GAUSSIAN_C :
* type : 임계점을 처리하는 방식
* block_size : 임계값을 적용할 영역의 크기
* C : 평균이나 가중 평균에서 차감할 값

cv2.adaptiveThreshold(image, max_value, adaptive_method, type, block_size, C):

### ADAPTIVE_THRESH_MEAN_C : 주변영역의 평균값으로 결정
- 적용할 픽셀(x,y)를 중심으로 하는 Block Size x Block Size 안에 있는  
  픽셀 값의 평균에서 C를 뺀 값을 임계점으로 사용
### ADAPTIVE_THRESH_GAUSSIAN_C
- 적용할 픽셀(x,y)를 중심으로 하는 Block Size x Block Size 안에 있는  
  Gaussian 윈도우 기반의 가중치들의 합에서 C를 뺀 값을 임계점으로 사용
- 가운데가 가장 큰 값, 가장자리로 갈수록 작은 값





In [ ]:
import cv2

def empty(pos):
    # print(pos)
    pass

img = cv2.imread('./data/book.png', cv2.IMREAD_GRAYSCALE)

name = 'Trackbar'
cv2.namedWindow(name)

# bar 이름, 창의 이름, 초기값, 최대값, 이벤트 처리
cv2.createTrackbar('block_size', name, 25, 100, empty) #홀수만 가능, 1보다는 큰값
cv2.createTrackbar('c', name, 3, 10, empty) #일반적으로 양수의 값을 사용


while True:
    block_size = cv2.getTrackbarPos('block_size', name) # bar이름, 창의 이름
    c = cv2.getTrackbarPos('c', name)

    if block_size <= 1: #1이하면 3으로
        block_size = 3

    if block_size % 2 == 0: #짝수이면 홀수로
        block_size += 1

    binary = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)

    cv2.imshow(name, binary)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

## 오츠 알고리즘

### Bimodal Image에 사용하기 적합(최적의 임계치를 자동으로 발견)
- Bimodal Image란 히스토그램 상에서 2개의 봉우리가 높게 치솟는 이미지  
```python
경계값을 임의로 정해서 픽셀들을 두 부류로 나누고, 두 부류의 명암 분포를 반복해서 구한 다음
두 부류의 명암 분포를 가장 균일하게 하는 경계값을 선택, 임계값 T라고 했을 때,
T를 기준으로 이진 분류된 픽셀의 비율의 차가 가장 작은 optimal T를 구하는 방식
```
- 모든 임계값을 다 걔산하기 때문에 속도가 느림
- 노이즈가 많은 영상에는 오츠 알고리즘을 적용해도 좋은 결과를 얻지 못함

In [1]:
import cv2
img = cv2.imread('./data/book.png', cv2.IMREAD_GRAYSCALE)

ret, otsu = cv2.threshold(img, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU,)
print('otsh threshold', ret)

cv2.imshow('img', img)
cv2.imshow('otsu', otsu)
cv2.waitKey(0)
cv2.destroyAllWindows()

otsh threshold 110.0
